In [1]:
import os

def print_tree(directory, indent=0):
    # List all files and directories in the current directory
    items = sorted(os.listdir(directory))
    for item in items:
        item_path = os.path.join(directory, item)
        if os.path.isdir(item_path):
            # Print directory name with indentation
            print("    " * indent + f" {item}")
            # Recursive call for subdirectories
            print_tree(item_path, indent + 1)
        else:
            # Print file name with indentation
            print("    " * indent + f" {item}")

# Specify the path to the 'vmd_output' folder
vmd_output_path = r"D:\AMRITA\Paper_SPS\Cardiorespiratory DB\vmd_output"
print(f"File structure of '{vmd_output_path}':")
print_tree(vmd_output_path)


File structure of 'D:\AMRITA\Paper_SPS\Cardiorespiratory DB\vmd_output':
 ECG10
     Breath_Hold
         Subject_1.0
             Segment_1_vmd.npy
             Segment_1_vmd.png
             Segment_2_vmd.npy
             Segment_2_vmd.png
             Segment_3_vmd.npy
             Segment_3_vmd.png
             Segment_4_vmd.npy
             Segment_4_vmd.png
             Segment_5_vmd.npy
             Segment_5_vmd.png
         Subject_10.0
             Segment_1_vmd.npy
             Segment_1_vmd.png
             Segment_2_vmd.npy
             Segment_2_vmd.png
             Segment_3_vmd.npy
             Segment_3_vmd.png
             Segment_4_vmd.npy
             Segment_4_vmd.png
             Segment_5_vmd.npy
             Segment_5_vmd.png
         Subject_11.0
             Segment_1_vmd.npy
             Segment_1_vmd.png
             Segment_2_vmd.npy
             Segment_2_vmd.png
             Segment_3_vmd.npy
             Segment_3_vmd.png
             Segment_4_vmd.npy
 

In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch


In [ ]:
# Function to load all signals and labels from multiple datasets
def load_multiple_datasets(base_dir):
    signals = []
    labels = []
    label_map = {"Breath_Hold": 0, "Normal_Breathing": 1, "Deep_Breathing": 2}  # Adjust as needed

    # Traverse each dataset (e.g., ECG6, ECG10)
    for dataset_name in os.listdir(base_dir):
        dataset_dir = os.path.join(base_dir, dataset_name)
        if not os.path.isdir(dataset_dir):
            continue

        # Traverse conditions within each dataset (e.g., Breath_Hold, Deep_Breathing)
        for label_name in os.listdir(dataset_dir):
            label_dir = os.path.join(dataset_dir, label_name)
            if not os.path.isdir(label_dir):
                continue

            # Traverse .npy files for each condition
            for root, _, files in os.walk(label_dir):
                for file in files:
                    if file.endswith(".npy"):
                        file_path = os.path.join(root, file)
                        try:
                            signal = np.load(file_path)
                            signals.append(signal)  # Keep signals as a list
                            labels.append(label_map[label_name])
                        except Exception as e:
                            print(f"Error loading file {file_path}: {e}")
                            continue

    if len(signals) == 0 or len(labels) == 0:
        print("No data loaded. Please check the data directory and ensure it contains .npy files.")
        raise ValueError("No data loaded. Please check the data directory and ensure it contains .npy files.")

    return signals, labels  # Return lists for variable-length handling


# Define a custom dataset class
class SignalDataset(Dataset):
    def __init__(self, signals, labels):
        self.signals = signals
        self.labels = labels

    def __len__(self):
        return len(self.signals)

    def __getitem__(self, idx):
        signal = torch.tensor(self.signals[idx], dtype=torch.float32)  # Convert to tensor
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return signal, label


from torch.nn.utils.rnn import pad_sequence

# Custom collate function for variable-length signals
def collate_fn(batch):
    signals, labels = zip(*batch)  # Separate signals and labels
    padded_signals = pad_sequence([torch.tensor(s, dtype=torch.float32) for s in signals], batch_first=True)
    return padded_signals, torch.tensor(labels)


# Main code
base_dir = r"D:\AMRITA\Paper_SPS\Cardiorespiratory DB\vmd_output"  # Base directory containing ECG6, ECG10, etc.

# Load data from all datasets
signals, labels = load_multiple_datasets(base_dir)

# Confirm data is loaded
print(f"Loaded {len(signals)} signals and {len(labels)} labels.")

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(signals, labels, test_size=0.2, random_state=42)

# Create Dataset objects
train_dataset = SignalDataset(X_train, y_train)
test_dataset = SignalDataset(X_test, y_test)

# Create DataLoader objects
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

# Verify shapes
for batch_idx, (signals_batch, labels_batch) in enumerate(train_loader):
    print(f"Batch {batch_idx}:")
    print(f"Number of signals: {len(signals_batch)}")
    print(f"Shapes of signals in this batch: {[s.shape for s in signals_batch]}")
    print(f"Labels batch shape: {labels_batch.shape}")
    break  # Inspect one batch

In [ ]:
print(labels)

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm

# Preprocessing Function: Normalize signals
def preprocess_signals(signals):
    normalized_signals = []
    for signal in signals:
        if np.std(signal) != 0:  # Avoid division by zero
            signal = (signal - np.mean(signal)) / np.std(signal)
        else:
            signal = signal - np.mean(signal)  # If variance is zero, just subtract the mean
        normalized_signals.append(signal)
    return normalized_signals

# 1D CNN Model
class RespiratorySignalClassifier(nn.Module):
    def __init__(self):
        super(RespiratorySignalClassifier, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = None  # Placeholder for dynamically determined input size
        self.fc2 = nn.Linear(128, 3)  # 3 output classes
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        if self.fc1 is None:  # Dynamically initialize fc1 during the first forward pass
            in_features = x.size(1) * x.size(2)
            self.fc1 = nn.Linear(in_features, 128).to(x.device)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        # Training loop
        train_loop = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Training", leave=False)
        for signals_batch, labels_batch in train_loop:
            try:
                # Move padded tensor to the device
                signals_batch = signals_batch.to(device, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
                labels_batch = labels_batch.to(device, dtype=torch.long)

                optimizer.zero_grad()
                outputs = model(signals_batch)
                loss = criterion(outputs, labels_batch)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                train_loop.set_postfix(loss=loss.item())
            except Exception as e:
                print(f"Error in training loop: {e}")


        # Validation loop
        val_loss, val_acc = validate_model(model, val_loader, criterion)
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}")

    return model

# Validation Function
def validate_model(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    val_loop = tqdm(val_loader, desc="Validating", leave=False)
    with torch.no_grad():
        for signals_batch, labels_batch in val_loop:
            try:
                signals_batch = signals_batch.to(device, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
                labels_batch = labels_batch.to(device, dtype=torch.long)

                outputs = model(signals_batch)
                loss = criterion(outputs, labels_batch)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total += labels_batch.size(0)
                correct += (predicted == labels_batch).sum().item()

                val_loop.set_postfix(loss=loss.item())
            except Exception as e:
                print(f"Error in validation loop: {e}")

    val_acc = correct / total
    return val_loss / len(val_loader), val_acc

# Device Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define model, criterion, and optimizer
model = RespiratorySignalClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
trained_model = train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=20)

# Test the model
model.eval()
all_predictions = []
all_labels = []
with torch.no_grad():
    for signals_batch, labels_batch in tqdm(test_loader, desc="Testing"):
        signals_batch = signals_batch.to(device, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
        labels_batch = labels_batch.to(device, dtype=torch.long)

        outputs = model(signals_batch)
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels_batch.cpu().numpy())

# Evaluate performance
print(classification_report(all_labels, all_predictions, target_names=["Breath_Hold", "Normal_Breathing", "Deep_Breathing"]))
print(f"Test Accuracy: {accuracy_score(all_labels, all_predictions):.4f}")


Epoch 1/20 - Training:   0%|          | 2/996 [00:00<01:12, 13.69it/s]

Error in training loop: 'list' object has no attribute 'to'
Error in training loop: 'list' object has no attribute 'to'
Error in training loop: 'list' object has no attribute 'to'
Error in training loop: 'list' object has no attribute 'to'
Error in training loop: 'list' object has no attribute 'to'


Epoch 1/20 - Training:   1%|          | 9/996 [00:00<00:40, 24.55it/s]

Error in training loop: 'list' object has no attribute 'to'
Error in training loop: 'list' object has no attribute 'to'
Error in training loop: 'list' object has no attribute 'to'
Error in training loop: 'list' object has no attribute 'to'
Error in training loop: 'list' object has no attribute 'to'
Error in training loop: 'list' object has no attribute 'to'
Error in training loop: 'list' object has no attribute 'to'


Epoch 1/20 - Training:   2%|▏         | 16/996 [00:00<00:42, 23.07it/s]

...

Epoch 1/20 - Training:   7%|▋         | 69/996 [00:04<01:27, 10.54it/s]

Error in training loop: 'list' object has no attribute 'to'
Error in training loop: 'list' object has no attribute 'to'
Error in training loop: 'list' object has no attribute 'to'


Epoch 1/20 - Training:   7%|▋         | 73/996 [00:04<01:13, 12.60it/s]

Error in training loop: 'list' object has no attribute 'to'
Error in training loop: 'list' object has no attribute 'to'
Error in training loop: 'list' object has no attribute 'to'
Error in training loop: 'list' object has no attribute 'to'


KeyboardInterrupt: 